In [1]:
# default_exp algo.ml.tree.lgb

# LightGBM, Light Gradient Boosting Machine

https://github.com/microsoft/LightGBM

LightGBM是使用基于树的学习算法的梯度增强框架。 它被设计为分布式且高效的，具有以下优点：

     训练速度更快，效率更高。

     降低内存使用率。

     更好的准确性。

     支持并行和GPU学习。

     能够处理大规模数据。

## install

In [3]:
# !pip install lightgbm -U
!pip freeze | grep lightgbm

lightgbm==2.2.3


# python guide
https://github.com/microsoft/LightGBM/tree/master/examples/python-guide

## simple_example
https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/simple_example.py


    Construct Dataset
    Basic train and predict
    Eval during training
    Early stopping
    Save model to file


In [4]:
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error

/Users/luoyonggui/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:47: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [5]:
data_dir = '../data/lgb_data/'

In [6]:
print('Loading data...')
# load or create your dataset
df_train = pd.read_csv(data_dir + 'regression.train', header=None, sep='\t')
df_test = pd.read_csv(data_dir + 'regression.test', header=None, sep='\t')

Loading data...


In [7]:
y_train = df_train[0]
y_test = df_test[0]
X_train = df_train.drop(0, axis=1)
X_test = df_test.drop(0, axis=1)

# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [8]:
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

Starting training...
[1]	valid_0's l2: 0.243898	valid_0's l1: 0.492841
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.240605	valid_0's l1: 0.489327
[3]	valid_0's l2: 0.236472	valid_0's l1: 0.484931
[4]	valid_0's l2: 0.232586	valid_0's l1: 0.480567
[5]	valid_0's l2: 0.22865	valid_0's l1: 0.475965
[6]	valid_0's l2: 0.226187	valid_0's l1: 0.472861
[7]	valid_0's l2: 0.223738	valid_0's l1: 0.469847
[8]	valid_0's l2: 0.221012	valid_0's l1: 0.466258
[9]	valid_0's l2: 0.218429	valid_0's l1: 0.462751
[10]	valid_0's l2: 0.215505	valid_0's l1: 0.458755
[11]	valid_0's l2: 0.213027	valid_0's l1: 0.455252
[12]	valid_0's l2: 0.210809	valid_0's l1: 0.452051
[13]	valid_0's l2: 0.208612	valid_0's l1: 0.448764
[14]	valid_0's l2: 0.207468	valid_0's l1: 0.446667
[15]	valid_0's l2: 0.206009	valid_0's l1: 0.444211
[16]	valid_0's l2: 0.20465	valid_0's l1: 0.44186
[17]	valid_0's l2: 0.202489	valid_0's l1: 0.438508
[18]	valid_0's l2: 0.200668	valid_0's l1: 0.435919
[19]	valid_0

## sklearn_example
https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/sklearn_example.py


    Create data for learning with sklearn interface
    Basic train and predict with sklearn interface
    Feature importances with sklearn interface
    Self-defined eval metric with sklearn interface
    Find best parameters for the model with sklearn's GridSearchCV


In [9]:
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [10]:
data_dir = '../data/lgb_data/'

In [11]:
print('Loading data...')
# load or create your dataset
df_train = pd.read_csv(data_dir + 'regression.train', header=None, sep='\t')
df_test = pd.read_csv(data_dir + 'regression.test', header=None, sep='\t')

Loading data...


In [13]:
df_train.head(2)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1,0.869,-0.635,0.226,0.327,-0.690,0.754,-0.249,-1.092,0.000,...,-0.010,-0.046,3.102,1.354,0.980,0.978,0.920,0.722,0.989,0.877
1,1,0.908,0.329,0.359,1.498,-0.313,1.096,-0.558,-1.588,2.173,...,-1.139,-0.001,0.000,0.302,0.833,0.986,0.978,0.780,0.992,0.798


In [14]:
df_test.head(2)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1,0.644,0.247,-0.447,0.862,0.374,0.854,-1.126,-0.790,2.173,...,-0.19,-0.744,3.102,0.958,1.061,0.980,0.875,0.581,0.905,0.796
1,0,0.385,1.800,1.037,1.044,0.349,1.502,-0.966,1.734,0.000,...,-0.44,0.638,3.102,0.695,0.909,0.981,0.803,0.813,1.149,1.116


In [15]:
y_train = df_train[0]
y_test = df_test[0]
X_train = df_train.drop(0, axis=1)
X_test = df_test.drop(0, axis=1)

In [16]:
print('Starting training...')
# train
gbm = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5)

Starting training...
[1]	valid_0's l2: 0.242763	valid_0's l1: 0.491735
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.237895	valid_0's l1: 0.486563
[3]	valid_0's l2: 0.233277	valid_0's l1: 0.481489
[4]	valid_0's l2: 0.22925	valid_0's l1: 0.476848
[5]	valid_0's l2: 0.226155	valid_0's l1: 0.47305
[6]	valid_0's l2: 0.222963	valid_0's l1: 0.469049
[7]	valid_0's l2: 0.220364	valid_0's l1: 0.465556
[8]	valid_0's l2: 0.217872	valid_0's l1: 0.462208
[9]	valid_0's l2: 0.215328	valid_0's l1: 0.458676
[10]	valid_0's l2: 0.212743	valid_0's l1: 0.454998
[11]	valid_0's l2: 0.210805	valid_0's l1: 0.452047
[12]	valid_0's l2: 0.208945	valid_0's l1: 0.449158
[13]	valid_0's l2: 0.206986	valid_0's l1: 0.44608
[14]	valid_0's l2: 0.205513	valid_0's l1: 0.443554
[15]	valid_0's l2: 0.203728	valid_0's l1: 0.440643
[16]	valid_0's l2: 0.201865	valid_0's l1: 0.437687
[17]	valid_0's l2: 0.200639	valid_0's l1: 0.435454
[18]	valid_0's l2: 0.199522	valid_0's l1: 0.433288
[19]	valid_0

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=20, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [18]:
print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
y_pred[:5]

Starting predicting...


array([0.68205502, 0.4812164 , 0.37969294, 0.45091878, 0.38236335])

In [19]:
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

The rmse of prediction is: 0.4441153344254208


In [20]:
# feature importances
print('Feature importances:', list(gbm.feature_importances_))

Feature importances: [23, 7, 0, 33, 5, 56, 9, 1, 1, 21, 2, 5, 1, 19, 9, 6, 1, 10, 4, 10, 0, 31, 61, 4, 48, 102, 52, 79]


### self-defined eval metric

In [21]:
# self-defined eval metric
# f(y_true: array, y_pred: array) -> name: string, eval_result: float, is_higher_better: bool
# Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False


print('Starting training with custom eval function...')
# train
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=rmsle,
        early_stopping_rounds=5)

Starting training with custom eval function...
[1]	valid_0's l2: 0.242763	valid_0's RMSLE: 0.344957
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.237895	valid_0's RMSLE: 0.341693
[3]	valid_0's l2: 0.233277	valid_0's RMSLE: 0.338462
[4]	valid_0's l2: 0.22925	valid_0's RMSLE: 0.335656
[5]	valid_0's l2: 0.226155	valid_0's RMSLE: 0.333431
[6]	valid_0's l2: 0.222963	valid_0's RMSLE: 0.331104
[7]	valid_0's l2: 0.220364	valid_0's RMSLE: 0.329193
[8]	valid_0's l2: 0.217872	valid_0's RMSLE: 0.327337
[9]	valid_0's l2: 0.215328	valid_0's RMSLE: 0.325433
[10]	valid_0's l2: 0.212743	valid_0's RMSLE: 0.323523
[11]	valid_0's l2: 0.210805	valid_0's RMSLE: 0.321986
[12]	valid_0's l2: 0.208945	valid_0's RMSLE: 0.320523
[13]	valid_0's l2: 0.206986	valid_0's RMSLE: 0.319027
[14]	valid_0's l2: 0.205513	valid_0's RMSLE: 0.317796
[15]	valid_0's l2: 0.203728	valid_0's RMSLE: 0.316383
[16]	valid_0's l2: 0.201865	valid_0's RMSLE: 0.314827
[17]	valid_0's l2: 0.200639	valid_0's 

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=20, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [22]:
# another self-defined eval metric
# f(y_true: array, y_pred: array) -> name: string, eval_result: float, is_higher_better: bool
# Relative Absolute Error (RAE)
def rae(y_true, y_pred):
    return 'RAE', np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(np.mean(y_true) - y_true)), False


print('Starting training with multiple custom eval functions...')
# train
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=lambda y_true, y_pred: [rmsle(y_true, y_pred), rae(y_true, y_pred)],
        early_stopping_rounds=5)


Starting training with multiple custom eval functions...
[1]	valid_0's l2: 0.242763	valid_0's RMSLE: 0.344957	valid_0's RAE: 0.991146
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's l2: 0.237895	valid_0's RMSLE: 0.341693	valid_0's RAE: 0.98072
[3]	valid_0's l2: 0.233277	valid_0's RMSLE: 0.338462	valid_0's RAE: 0.970493
[4]	valid_0's l2: 0.22925	valid_0's RMSLE: 0.335656	valid_0's RAE: 0.961139
[5]	valid_0's l2: 0.226155	valid_0's RMSLE: 0.333431	valid_0's RAE: 0.953484
[6]	valid_0's l2: 0.222963	valid_0's RMSLE: 0.331104	valid_0's RAE: 0.945419
[7]	valid_0's l2: 0.220364	valid_0's RMSLE: 0.329193	valid_0's RAE: 0.938379
[8]	valid_0's l2: 0.217872	valid_0's RMSLE: 0.327337	valid_0's RAE: 0.931631
[9]	valid_0's l2: 0.215328	valid_0's RMSLE: 0.325433	valid_0's RAE: 0.92451
[10]	valid_0's l2: 0.212743	valid_0's RMSLE: 0.323523	valid_0's RAE: 0.917099
[11]	valid_0's l2: 0.210805	valid_0's RMSLE: 0.321986	valid_0's RAE: 0.911151
[12]	valid_0's l2: 0.208945	valid_0'

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=20, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [23]:
print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
# eval
print('The rmsle of prediction is:', rmsle(y_test, y_pred)[1])
print('The rae of prediction is:', rae(y_test, y_pred)[1])

Starting predicting...
The rmsle of prediction is: 0.3110323289863278
The rae of prediction is: 0.8645881044669875


In [24]:
# other scikit-learn modules
estimator = lgb.LGBMRegressor(num_leaves=31)

param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}

gbm = GridSearchCV(estimator, param_grid, cv=3)
gbm.fit(X_train, y_train)

print('Best parameters found by grid search are:', gbm.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.1, 'n_estimators': 40}


## advanced_example
https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/advanced_example.py


    Construct Dataset
    Set feature names
    Directly use categorical features without one-hot encoding
    Save model to file
    Dump model to JSON format
    Get feature names
    Get feature importances
    Load model to predict
    Dump and load model with pickle
    Load model file to continue training
    Change learning rates during training
    Change any parameters during training
    Self-defined objective function
    Self-defined eval metric
    Callback function


# nb_export

In [4]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 00_template.ipynb.
Converted algo_ml_shallow_tree_catboost.ipynb.
Converted dl_keras.ipynb.
Converted engineering_nbdev.ipynb.
Converted engineering_panel.ipynb.
Converted index.ipynb.
